# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Introduction : Business Problem

In this project I will try to find the best place for a restaurant. This report will be targeted to stakeholders who will open a restaurant in Toronto, Canada.

Since there are lots of restaurants in Toronto I will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Italian restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data

Based on definition of our problem, factors that will influence our decission are:

number of existing restaurants in the neighborhood (any type of restaurant)
distance to city center, if the number of locations with the minimum number of shops is multiple

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

number of restaurants nearby the city will be collected by Foursquare API

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# create map of New York using latitude and longitude values
latitude=43.65
longitude=-79.38
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto


In [3]:
# add markers to map
location=[]
for i in range(-5, 5,1):
    for j in range(-5,5,1):
        location.append((latitude+i*0.001,longitude+j*0.001))
for lat, lng in location:
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    print(1,end=' ')
map_toronto

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [4]:
CLIENT_ID = 'Q21MUMFWLDCZTZ1M5AAZU2V0TK3150K20R5O2GLNAPX51PTL' # your Foursquare ID
CLIENT_SECRET = 'QICL4M4RACDASB55VM3R3QRE4ZR0RNL1VKWWZ0EHF3M5HREX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q21MUMFWLDCZTZ1M5AAZU2V0TK3150K20R5O2GLNAPX51PTL
CLIENT_SECRET:QICL4M4RACDASB55VM3R3QRE4ZR0RNL1VKWWZ0EHF3M5HREX


In [5]:
import json

In [6]:
LIMIT=100
radius=300
def getNearbyVenues(location, radius=300):
    
    venues_list=[]
    for lat, lng in location:
        name=(lat,lng)
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
toronto_venues = getNearbyVenues(location)

(43.644999999999996, -79.38499999999999)
(43.644999999999996, -79.384)
(43.644999999999996, -79.383)
(43.644999999999996, -79.38199999999999)
(43.644999999999996, -79.381)
(43.644999999999996, -79.38)
(43.644999999999996, -79.37899999999999)
(43.644999999999996, -79.378)
(43.644999999999996, -79.377)
(43.644999999999996, -79.37599999999999)
(43.646, -79.38499999999999)
(43.646, -79.384)
(43.646, -79.383)
(43.646, -79.38199999999999)
(43.646, -79.381)
(43.646, -79.38)
(43.646, -79.37899999999999)
(43.646, -79.378)
(43.646, -79.377)
(43.646, -79.37599999999999)
(43.647, -79.38499999999999)
(43.647, -79.384)
(43.647, -79.383)
(43.647, -79.38199999999999)
(43.647, -79.381)
(43.647, -79.38)
(43.647, -79.37899999999999)
(43.647, -79.378)
(43.647, -79.377)
(43.647, -79.37599999999999)
(43.647999999999996, -79.38499999999999)
(43.647999999999996, -79.384)
(43.647999999999996, -79.383)
(43.647999999999996, -79.38199999999999)
(43.647999999999996, -79.381)
(43.647999999999996, -79.38)
(43.647999

In [8]:
print(toronto_venues.shape)
toronto_venues.head()

(5262, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"(43.644999999999996, -79.38499999999999)",43.645,-79.385,Union Pearson Express,43.644362,-79.383199,Train Station
1,"(43.644999999999996, -79.38499999999999)",43.645,-79.385,Roy Thomson Hall,43.646589,-79.385979,Concert Hall
2,"(43.644999999999996, -79.38499999999999)",43.645,-79.385,The Ritz-Carlton,43.645330,-79.387089,Hotel
3,"(43.644999999999996, -79.38499999999999)",43.645,-79.385,Delta Hotels by Marriott Toronto,43.642882,-79.383949,Hotel
4,"(43.644999999999996, -79.38499999999999)",43.645,-79.385,Starbucks,43.646731,-79.383951,Coffee Shop


In [9]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"(43.644999999999996, -79.38499999999999)",15,15,15,15,15,15
"(43.644999999999996, -79.384)",18,18,18,18,18,18
"(43.644999999999996, -79.383)",18,18,18,18,18,18
"(43.644999999999996, -79.38199999999999)",65,65,65,65,65,65
"(43.644999999999996, -79.381)",61,61,61,61,61,61
"(43.644999999999996, -79.38)",55,55,55,55,55,55
"(43.644999999999996, -79.37899999999999)",47,47,47,47,47,47
"(43.644999999999996, -79.378)",39,39,39,39,39,39
"(43.644999999999996, -79.377)",14,14,14,14,14,14


In [10]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 129 uniques categories.


In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poke Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(43.644999999999996, -79.38499999999999)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(43.644999999999996, -79.38499999999999)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(43.644999999999996, -79.38499999999999)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(43.644999999999996, -79.38499999999999)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(43.644999999999996, -79.38499999999999)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
toronto_onehot.shape

(5262, 129)

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poke Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store
0,"(43.644999999999996, -79.38499999999999)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.266667,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"(43.644999999999996, -79.384)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.277778,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [52]:
toronto_grouped.shape

(39, 234)

In [18]:
L=[]
min=1
for hood in toronto_grouped['Neighborhood']:
    print(hood)
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood]['American Restaurant']
    print(temp)
    temp=float(temp)
    if min>temp:
        min=temp
        L=[hood]
    elif min==temp:
        L.append(hood)

(43.644999999999996, -79.38499999999999)
0    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.384)
1    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.383)
2    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.38199999999999)
3    0.015385
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.381)
4    0.016393
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.38)
5    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.37899999999999)
6    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.378)
7    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.377)
8    0.0
Name: American Restaurant, dtype: float64
(43.644999999999996, -79.37599999999999)
9    0.0
Name: American Restaurant, dtype: float64
(43.646, -79.38499999999999)
10    0.0
Name: American Restaurant, dtype: float64
(43.646, -79.384)
11    0.0
Name: American Restaurant, d

In [20]:
min=10
for lat,lng in L:
    dis=(latitude-lat)**2+(longitude-lng)**2
    if dis<min:
        place=(lat,lng)
        min=dis
print(place)

(43.65, -79.37599999999999)
